# Read Initial Data

In [54]:
import pandas as pd
time = []
open = []
high = []
low = []
close = []
volume = []
size = -1

def read_df(file_name):

    df = pd.read_feather(file_name)
    df = df.dropna()    # Dropping NA values, so that talib works correctly
    global size
    size = df.shape[0]
    
    global time
    time = df['time'].to_numpy()

    global open
    open = df['open'].to_numpy()

    global high
    high = df['high'].to_numpy()

    global low
    low = df['low'].to_numpy()

    global close
    close = df['close'].to_numpy()

    global volume
    volume = df['volume'].to_numpy()

    return

read_df("data-BNBUSDT1h.feather")

# Create Indicators

In [55]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import talib as ta    # https://github.com/mrjbq7/ta-lib

ma_direction = ta.EMA(close, timeperiod=28)
atr_direction = ta.ATR(high, low, close, timeperiod=14)

# Generate Trades

In [56]:
in_trade = 0    # +1 = long_entry, -1 = shor_entry

entry_price = -1.0
trades_long = []
trades_short = []

for i in range(size):
    # Long Entry
    if (ma_direction[i] - close[i] > 2.0*atr_direction[i]
        and in_trade == 0.0):
        in_trade = +1.0
        entry_price = close[i]
    # Long Exit
    if (ma_direction[i] - close[i] < 0.0
        and in_trade == 1.0):
        in_trade = 0.0
        trades_long.append(close[i]/entry_price)

    # Short Entry
    if (ma_direction[i] - close[i] < -2.0*atr_direction[i]
        and in_trade == 0.0):
        in_trade = -1.0
        entry_price = close[i]
    # Short Exit
    if (ma_direction[i] - close[i] > 0.0
        and in_trade == -1.0):
        in_trade = 0.0
        trades_short.append(entry_price/close[i])


# Calculate Results

In [57]:
long_res = 1.0
for k in trades_long:
    long_res *= k
print("Longs result: ", long_res)
print("Average Long Trade: ", np.sum(trades_long)/len(trades_long))
print()

short_res = 1.0
for k in trades_short:
    short_res *= k
print("Shorts result: ", short_res)
print("Average Short Trade: ", np.sum(trades_short)/len(trades_short))

Longs result:  0.5269678534329184
Average Long Trade:  0.9996827025941656

Shorts result:  0.027721723092249302
Average Short Trade:  0.9932448041098874


# Optimize Results